# Mining Function Specifications

When testing a program, one not only needs to cover its several behaviors; one also needs to _check_ whether the result is as expected.  In this chapter, we introduce a technique that allows us to _mine_ function specifications from a set of given executions, resulting in abstract and formal _descriptions_ of what the function expects and what it delivers.  

These so-called _dynamic invariants_ produce pre- and post-conditions over function arguments and variables from a set of executions.  They are useful in a variety of contexts:

* Dynamic invariants provide important information for [symbolic fuzzing](SymbolicFuzzing.ipynb), such as types and ranges of function arguments.
* Dynamic invariants provide pre- and postconditions for formal program proofs and verification.
* Dynamic invariants provide a large number of assertions that can check whether function behavior has changed.

Traditionally, dynamic invariants are dependent on the executions they are derived from.  However, when paired with comprehensive test generators, they quickly become very precise, as we show in this chapter.

**Prerequisites**

* You should be familiar with tracing program executions, as in the [chapter on coverage](Coverage.ipynb).
* Later in this section, we make use of Python program transformations; some knowledge on the Python AST functions is helpful.
* The interplay with symbolic testing builds on, well, [symbolic testing](SymbolicFuzzer.ipynb),

In [ ]:
import fuzzingbook_utils

In [ ]:
import Intro_Testing

## Specifications and Assertions

When implementing a function or program, one usually works against a _specification_ – a set of documented requirements to be satisfied by the code.  Such specifications can come in natural language.  A formal specification, however, allows the computer to check whether the specification is satisfied.

In the [introduction to testing](Intro_Testing.ipynb), we have seen how _preconditions_ and _postconditions_ can describe what a function does.  Consider the following (simple) square root function:

In [ ]:
def my_sqrt(x):
    assert x >= 0  # Precondition
    
    ...
    
    assert result * result == x  # Postcondition
    return result

The assertion `assert p` checks the condition `p`; if it does not hold, execution is aborted.  Here, the actual body is not yet written; we use the assertions as a specification of what `my_sqrt()` _expects_, and what it _delivers_.

The topmost assertion is the _precondition_, stating the requirements on the function arguments.  The assertion at the end is the _postcondition_, stating the properties of the function result (including its relationship with the original arguments).  Using these pre- and postconditions as a specification, we can now go and implement a square root function that satisfies them.  Once implemented, we can have the assertions check at runtime whether `my_sqrt()` works as expected; a [symbolic](SymbolicFuzzer.ipynb) or [concolic](ConcolicFuzzer.ipynb) test generator will even specifically try to find inputs where the assertions do _not_ hold.  (An assertion can be seen as a conditional branch towards aborting the execution, and any technique that tries to cover all code branches will also try to invalidate as many assertions as possible.)

However, not every piece of code is developed with explicit specifications in the first place; let alone does most code comes with formal pre- and post-conditions.  (Just take a look at the chapters in this book.)  This is a pity: As Ken Thompson famously said, "Without specifications, there are no bugs – only surprises".  It is also a problem for testing, since, of course, testing needs some specification to test against.  This raises the interesting question: Can we somehow _retrofit_ existing code with "specifications" that properly describe their behavior, allowing developers to simply _check_ them rather than having to write them from scratch?  This is what we do in this chapter.}

## Mining Type Specifications

For our Python code, one of the most important "specifications" we need is *types*.  Python being a "dynamically" typed language means that all data types are determined at run time; the code itself does not explicitly state whether a variable is an integer, a string, an array, a dictionary – or whatever.  As _writer_ of Python code, omitting explicit type declarations may save time (and allows for some fun hacks).  It is not sure whether a lack of types helps in _reading_ and _understanding_ code for humans.  For a _computer_ trying to analyze code, the lack of explicit types is detrimental.  If, say, a constraint solver, sees `if x:` and cannot know whether `x` is supposed to be a number or a string, this introduces an ambiguity which multiplies over the entire analysis in a combinatorial explosion.  Our first task thus will be to mine _static_ types (as part of the code) from _values_ we observe at run time.

How can we mine types from executions?  The answer is simple: 

1. We observe a function during execution
2. We track the _types_ of its arguments
3. We include these types as annotations or assertions into the codee.

To do so, we can make use of Python's tracing facility we already observed in the [chapter on coverage](Coverage.ipynb).  With every call to a function, we retrieve the arguments, their values, and their types.

As an example, consider the full implementation of `my_sqrt()` from the [introduction to testing](Intro_Testing.ipynb):

In [ ]:
import fuzzingbook_utils

In [ ]:
def my_sqrt(x):
    """Computes the square root of x, using the Newton-Raphson method"""
    approx = None
    guess = x / 2
    while approx != guess:
        approx = guess
        guess = (approx + x / approx) / 2
    return approx

`my_sqrt()` does not come with any assertions that would check types or values:

In [ ]:
from ExpectError import ExpectError, ExpectTimeout

In [ ]:
with ExpectError():
    my_sqrt("foo")

In [ ]:
with ExpectTimeout(1):
    x = my_sqrt(-1)

Our goal is to determine types from _normal_ invocations such as this one:

In [ ]:
my_sqrt(25)

### Tracking Calls

We can define a _tracer function_ that tracks the execution of `my_sqrt()`, listing its arguments and return values:

In [ ]:
import sys

In [ ]:
class Tracker(object):
    def __init__(self, log=False):
        self._log = log
        self.reset()

    def reset(self):
        self._calls = {}
        self._stack = []

    # Start of `with` block
    def __enter__(self):
        self.original_trace_function = sys.gettrace()
        sys.settrace(self.traceit)
        return self

    # End of `with` block
    def __exit__(self, exc_type, exc_value, tb):
        sys.settrace(self.original_trace_function)

In [ ]:
def get_arguments(frame):
    """Return call arguments in the given frame"""
    # When called, all arguments are local variables
    arguments = [(var, frame.f_locals[var]) for var in frame.f_locals]
    arguments.reverse()  # Want same order as call
    return arguments

In [ ]:
def simple_call_string(function_name, argument_list):
    """Return function_name(arg[0], arg[1], ...) as a string"""
    return function_name + "(" + \
        ", ".join([var + "=" + repr(value)
                   for (var, value) in argument_list]) + ")"

In [ ]:
class CallTracker(Tracker):
    def add_call(self, function_name, arguments, return_value=None):
        """Add given call to list of calls"""
        if function_name not in self._calls:
            self._calls[function_name] = []
        self._calls[function_name].append((arguments, return_value))

    # Tracking function: Record all calls and all args
    def traceit(self, frame, event, arg):
        if event == "call":
            self.trace_call(frame, event, arg)
        elif event == "return":
            self.trace_return(frame, event, arg)
            
        return self.traceit
        
    def trace_call(self, frame, event, arg):
        code = frame.f_code
        function_name = code.co_name
        arguments = get_arguments(frame)
        self._stack.append((function_name, arguments))

        if self._log:
            print(simple_call_string(function_name, arguments))

    def trace_return(self, frame, event, arg):
        code = frame.f_code
        function_name = code.co_name
        return_value = arg
        
        called_function_name, called_arguments = self._stack.pop()
        assert function_name == called_function_name
        
        if self._log:
            print(simple_call_string(function_name, called_arguments), "returns", return_value)
            
        self.add_call(function_name, called_arguments, return_value)

In [ ]:
class CallTracker(CallTracker):
    def calls(self, function_name=None):
        if function_name is None:
            return self._calls

        return self._calls[function_name]

In [ ]:
with CallTracker(log=True) as tracker:
    y = my_sqrt(25)
    y = my_sqrt(2.0)

In [ ]:
calls = tracker.calls('my_sqrt')
calls

In [ ]:
def simple_call_string(function_name, argument_list, return_value = None):
    """Return function_name(arg[0], arg[1], ...) as a string"""
    call = function_name + "(" + \
        ", ".join([var + "=" + repr(value)
                   for (var, value) in argument_list]) + ")"

    if return_value is not None:
        call += " = " + repr(return_value)
        
    return call

\todo{Import these into [Carver](Carver.ipynb)}

In [ ]:
argument_list, return_value = calls[0]
simple_call_string('my_sqrt', argument_list, return_value)

### Getting Types

Python has an elaborate type system:

In [ ]:
type(4)

In [ ]:
type(2.0)

In [ ]:
type([4])

We can retrieve the type of the first argument to `my_sqrt()`:

In [ ]:
parameter, value = argument_list[0]
parameter, type(value)

as well as the type of the return value:

In [ ]:
type(return_value)

Hence, we see that (so far), `my_sqrt()` is a function taking (among others) integers and returning floats.  We could declare `my_sqrt()` as:

In [ ]:
def my_sqrt_annotated(x: int) -> float:
    return my_sqrt(x)

This is a representation we could place in a static type checker, allowing to check whether calls to `my_sqrt()` actually pass a number.  A dynamic type checker could run such checks at runtime.  And of course, any symbolic interpretation will greatly profit from the additional annotations.

By default, Python does not do anything with such annotations.  However, tools can access annotations from functions and other objects:

In [ ]:
my_sqrt_annotated.__annotations__

### Annotating Functions

Let us go and annotate functions automatically, based on the types we have seen.

In [ ]:
import ast
import inspect
import astunparse

We can get the source of a function (if it is in the same file)

In [ ]:
from fuzzingbook_utils import print_content

In [ ]:
from pygments.lexers import PythonLexer

In [ ]:
my_sqrt_source = inspect.getsource(my_sqrt)
print_content(my_sqrt_source, '.py')

Parsing this gives us an abstract syntax tree (AST):

In [ ]:
my_sqrt_ast = ast.parse(my_sqrt_source)

In [ ]:
ast.dump(my_sqrt_ast)

Printing this out as Python code is a bit more readable:

In [ ]:
print(astunparse.unparse(my_sqrt_ast))

We want to transform this adding annotations.

First, a helper function to parse type names:

In [ ]:
def parse_type(name):
    class ValueVisitor(ast.NodeVisitor):
        def visit_Expr(self, node):
            self.value_node = node.value
        
    tree = ast.parse(name)
    name_visitor = ValueVisitor()
    name_visitor.visit(tree)
    return name_visitor.value_node

In [ ]:
ast.dump(parse_type('int'))

In [ ]:
ast.dump(parse_type('[Any]'))

Now, a helper to actually add type annotations to a function AST.  This would be called as

```python
    TypeTransformer({'x': 'int'}, 'float').visit(ast)
```

to annotate the arguments of `my_sqrt()`: `x` with `int`, and the return type with `float`.  The returned AST can then be unparsed, compiled or analyzed.

In [ ]:
class TypeTransformer(ast.NodeTransformer):
    def __init__(self, argument_types, return_type=None):
        self.argument_types = argument_types
        self.return_type = return_type
        super().__init__()

In [ ]:
class TypeTransformer(TypeTransformer):
    def annotate_arg(self, arg):
        """Add annotation to single function argument"""
        arg_name = arg.arg
        if arg_name in self.argument_types:
            arg.annotation = parse_type(self.argument_types[arg_name])
        return arg

In [ ]:
class TypeTransformer(TypeTransformer):
    def visit_FunctionDef(self, node):
        """Add annotation to function"""
        # Set argument types
        new_args = []
        for arg in node.args.args:
            new_args.append(self.annotate_arg(arg))

        new_arguments = ast.arguments(
            new_args,
            node.args.vararg,
            node.args.kwonlyargs,
            node.args.kw_defaults,
            node.args.kwarg,
            node.args.defaults
        )

        # Set return type
        node.returns = parse_type(self.return_type)
        
        return ast.copy_location(ast.FunctionDef(node.name, new_arguments, 
                                                 node.body, node.decorator_list,
                                                 node.returns), node)

Does this work?  Yes!

In [ ]:
new_ast = TypeTransformer({'x': 'int'}, 'float').visit(my_sqrt_ast)

In [ ]:
print_content(astunparse.unparse(new_ast), '.py')

### All Together



In [ ]:
with CallTracker() as tracker:
    y = my_sqrt(25)
    y = my_sqrt(2.0)

In [ ]:
tracker.calls()

In [ ]:
def type_string(value):
    return type(value).__name__

In [ ]:
type_string(4)

In [ ]:
type_string([])

In [ ]:
def annotate_types(calls):
    annotated_functions = {}
    
    for function_name in calls:
        try:
            annotated_functions[function_name] = annotate_function_with_types(function_name, calls[function_name])
        except KeyError:
            continue

    return annotated_functions

In [ ]:
def annotate_function_with_types(function_name, function_calls):
    function = globals()[function_name]
    function_code = inspect.getsource(function)
    function_ast = ast.parse(function_code)
    return annotate_function_ast_with_types(function_ast, function_calls)

In [ ]:
def annotate_function_ast_with_types(function_ast, function_calls):
    parameter_types = {}
    return_type = None
        
    for calls_seen in function_calls:
        args, return_value = calls_seen
        if return_value is not None:
            return_type = type_string(return_value)
        for parameter, value in args:
            parameter_types[parameter] = type_string(value)
        
    annotated_function_ast = TypeTransformer(parameter_types, return_type).visit(function_ast)
    return annotated_function_ast

In [ ]:
print_content(astunparse.unparse(annotate_types(tracker.calls())['my_sqrt']), '.py')

In [ ]:
class TypeTracker(CallTracker):
    pass

In [ ]:
class TypeAnnotator(TypeTracker):
    def typed_functions_ast(self, function_name=None):
        if function_name is None:
            return annotate_types(self.calls())
        
        return annotate_function_with_types(function_name, self.calls(function_name))
    
    def typed_functions(self):
        return ''.join([astunparse.unparse(self.typed_functions_ast(function_name))         
                            for function_name in self.calls()])

In [ ]:
with TypeAnnotator() as annotator:
    y = my_sqrt(25)
    y = my_sqrt(2.0)

In [ ]:
print_content(annotator.typed_functions(), '.py')

In [ ]:
def sum3(a, b, c):
    return a + b + c

In [ ]:
with TypeAnnotator() as annotator:
    y = my_sqrt(1.0)
    y = sum3(1.0, 2.0, 3.0)

In [ ]:
print_content(annotator.typed_functions(), '.py')

In [ ]:
with TypeAnnotator() as annotator:
    y = sum3("one", "two", "three")

In [ ]:
print_content(annotator.typed_functions(), '.py')

### Use in Symbolic Testing

\todo{add}

In [ ]:
import SymbolicFuzzer  # minor dependency

## Mining Invariants

Besides basic data types. we can infer several further properties from arguments.  We can, for instance, infer the _range_ of values a variable is in, and thus determine whether an argument can be negative, zero, or positive – a property that cannot be expressed in a (Python) type.  To this end, we use the same tracking functionality as before; instead of saving types for individual variables, we now save whether we have seen specific properties occurring.

the _smallest_ and _largest_ values observed.  For a function such as `sqrt(x)`, for instance, we could go and find that in actual executions, the smallest value of `x` is zero, while there is (apparently) no upper limit for `x`.

Once we have mined such ranges, we can use them as _pre- and postconditions_ in functions – that is, we can have functions check automatically whether values fall into the ranges observed earlier.  A `sqrt(x)` function, for instance, could check whether its argument `x` is within the range `x >= 0` observed earlier – that is, a _precondition_:

In [ ]:
def sqrt_with_precondition(x):
    assert x >= 0
    ...

For the _result_, a similar property should hold (postcondition)

In [ ]:
def sqrt_with_postcondition(x):
    return_value = ...
    assert return_value >= 0
    return return_value

### Defining Properties

In [ ]:
INVARIANT_PROPERTIES = [
    "X < 0",
    "X <= 0",
    "X > 0",
    "X >= 0",
    "X == 0",
    "X != 0",

    "X == Y",
    "X > Y",
    "X >= Y",
    "X <= Y",
    
    "isinstance(X, bool)",
    "isinstance(X, int)",
    "isinstance(X, float)",
    "isinstance(X, list)",
    "isinstance(X, dict)",
    
    "X == Y + Z",
    "X == Y * Z",
    "X == Y - Z",
    "X == Y / Z",

    "X < Y < Z",
    "X <= Y <= Z",
    "X > Y > Z",
    "X >= Y >= Z",

    "X == len(Y)",
    "X == sum(Y)",
    "X.startswith(Y)",
]

### Extracting Meta-Variables

In [ ]:
def metavars(prop):
    metavar_list = []
    
    class ArgVisitor(ast.NodeVisitor):
        def visit_Name(self, node):
            if node.id.isupper():
                metavar_list.append(node.id)

    ArgVisitor().visit(ast.parse(prop))
    return metavar_list

In [ ]:
assert metavars("X < 0") == ['X']

In [ ]:
assert metavars("X.startswith(Y)") == ['X', 'Y']

In [ ]:
assert metavars("isinstance(X, str)") == ['X']

### Instantiating Properties

In [ ]:
def instantiate_prop_ast(prop, var_names):
    class NameTransformer(ast.NodeTransformer):
        def visit_Name(self, node):
            if node.id not in mapping:
                return node
            return ast.Name(id=mapping[node.id], ctx=ast.Load())
    
    class BodyVisitor(ast.NodeVisitor):
        def visit_Lambda(self, node):
            self.body = node.body
    
    meta_variables = metavars(prop)
    assert len(meta_variables) == len(var_names)

    mapping = {}
    for i in range(0, len(meta_variables)):
        mapping[meta_variables[i]] = var_names[i]

    prop_ast = ast.parse(prop, mode='eval')
    new_ast = NameTransformer().visit(prop_ast)

    return new_ast

In [ ]:
def instantiate_prop(prop, var_names):
    prop_ast = instantiate_prop_ast(prop, var_names)
    return astunparse.unparse(prop_ast).strip()

In [ ]:
assert instantiate_prop("X > Y", ['a', 'b']) == '(a > b)'

In [ ]:
assert instantiate_prop("X.startswith(Y)", ['x', 'y']) == 'x.startswith(y)'

### Evaluating Properties

In [ ]:
def prop_function_text(prop):
    return "lambda " + ", ".join(metavars(prop)) + ": " + prop

In [ ]:
prop_function_text("X > Y")

In [ ]:
def prop_function_ast(prop):
    return ast.parse(prop_function_text(prop), mode='eval')

In [ ]:
prop_ast = prop_function_ast("X > Y")
ast.dump(prop_ast)

In [ ]:
def prop_function(prop):
    return eval(prop_function_text(prop))

In [ ]:
f = prop_function("X > Y")
f(100, 1)

### Extracting Invariants

In [ ]:
import itertools

In [ ]:
for combination in itertools.permutations(['a', 'b', 'c'], 2):
    print(combination)

In [ ]:
def true_property_instantiations(prop, vars_and_values, log=False):
    instantiations = set()
    p = prop_function(prop)

    len_metavars = len(metavars(prop))
    for combination in itertools.permutations(vars_and_values, len_metavars):
        args = [value for var_name, value in combination]
        var_names = [var_name for var_name, value in combination]
        
        try:
            result = p(*args)
        except:
            result = None

        if log:
            print(prop, combination, result)
        if result:
            instantiations.add((prop, tuple(var_names)))
            
    return instantiations

In [ ]:
true_property_instantiations("X < Y", [('x', -1), ('y', 1)], log=True)

In [ ]:
true_property_instantiations("X < 0", [('x', -1), ('y', 1)], log=True)

In [ ]:
class InvariantTracker(CallTracker):
    def __init__(self, props=None, **kwargs):
        if props is None:
            props = INVARIANT_PROPERTIES

        self.props = props
        super().__init__(**kwargs)

In [ ]:
class InvariantTracker(InvariantTracker):
    def invariants(self, function_name=None):
        if function_name is None:
            return {function_name: self.invariants(function_name) for function_name in self.calls()}
        
        invariants = None
        for variables, return_value in self.calls(function_name):
            vars_and_values = variables + [('return_value', return_value)]
            
            s = set()
            for prop in self.props:
                s |= true_property_instantiations(prop, vars_and_values, self._log)
            if invariants is None:
                invariants = s
            else:
                invariants &= s

        return invariants

In [ ]:
with InvariantTracker() as tracker:
    y = my_sqrt(25)
    y = my_sqrt(10)

tracker.calls()

In [ ]:
tracker.invariants()

In [ ]:
invs = tracker.invariants('my_sqrt')
invs

In [ ]:
def pretty_invariants(invariants):
    props = []
    for (prop, var_names) in invariants:
        props.append(instantiate_prop(prop, var_names))
    return sorted(props)

In [ ]:
pretty_invariants(invs)

In [ ]:
with InvariantTracker() as tracker:
    y = sum3(1, 2, 3)
    y = sum3(-4, -5, -6)
    
pretty_invariants(tracker.invariants('sum3'))

In [ ]:
with InvariantTracker() as tracker:
    y = sum3('a', 'b', 'c')
    y = sum3('f', 'e', 'd')
    
pretty_invariants(tracker.invariants('sum3'))

In [ ]:
with InvariantTracker() as tracker:
    y = sum3('a', 'b', 'c')
    y = sum3('c', 'b', 'a')
    y = sum3(-4, -5, -6)
    y = sum3(0, 0, 0)
    
pretty_invariants(tracker.invariants('sum3'))

### Annotating Functions with Invariants

In [ ]:
class InvariantAnnotator(InvariantTracker):
    def functions_with_invariants_ast(self, function_name=None):
        if function_name is None:
            return annotate_functions_with_invariants(self.invariants())
        
        return annotate_function_with_invariants(function_name, self.invariants(function_name))
    
    def functions_with_invariants(self, function_name=None):
        if function_name is None:
            functions = ''
            for f_name in self.invariants():
                try:
                    f_text = astunparse.unparse(self.functions_with_invariants_ast(f_name))
                except KeyError:
                    f_text = ''
                functions += f_text
            return functions

        return astunparse.unparse(self.functions_with_invariants_ast(function_name))

In [ ]:
def annotate_invariants(invariants):
    annotated_functions = {}
    
    for function_name in invariants:
        try:
            annotated_functions[function_name] = annotate_function_with_invariants(function_name, invariants[function_name])
        except KeyError:
            continue

    return annotated_functions

In [ ]:
def annotate_function_with_invariants(function_name, function_invariants):
    function = globals()[function_name]
    function_code = inspect.getsource(function)
    function_ast = ast.parse(function_code)
    return annotate_function_ast_with_invariants(function_ast, function_invariants)

In [ ]:
def annotate_function_ast_with_invariants(function_ast, function_invariants):
    annotated_function_ast = InvariantTransformer(function_invariants).visit(function_ast)
    return annotated_function_ast

#### Preconditions

In [ ]:
class PreconditionTransformer(ast.NodeTransformer):
    def __init__(self, invariants):
        self.invariants = invariants
        super().__init__()
        
    def preconditions(self):
        preconditions = []
        for (prop, var_names) in self.invariants:
            assertion = "assert " + instantiate_prop(prop, var_names) + ', "violated precondition"'
            assertion_ast = ast.parse(assertion)

            if assertion.find('return_value') < 0:
                preconditions += assertion_ast.body

        return preconditions
    
    def insert_assertions(self, body):
        preconditions = self.preconditions()
        try:
            docstring = body[0].value.s
        except:
            docstring = None
            
        if docstring:
            return [body[0]] + preconditions + body[1:]
        else:
            return preconditions + body

    def visit_FunctionDef(self, node):
        """Add invariants to function"""
        # print(ast.dump(node))
        node.body = self.insert_assertions(node.body)
        return node    

In [ ]:
class InvariantTransformer(PreconditionTransformer):
    pass

In [ ]:
with InvariantAnnotator() as annotator:
    my_sqrt(5)

In [ ]:
print_content(annotator.functions_with_invariants(), '.py')

In [ ]:
with InvariantAnnotator() as annotator:
    y = sum3(3, 4, 5)
    y = sum3(-3, -4, -5)
    y = sum3(0, 0, 0)

In [ ]:
print_content(annotator.functions_with_invariants(), '.py')

#### Postconditions

We make a few simplifying assumptions: 

* Variables do not change during execution.
* There is a single `return` statement at the end of the function.

In [ ]:
class InvariantTransformer(PreconditionTransformer):
    def postconditions(self):
        postconditions = []

        for (prop, var_names) in self.invariants:
            assertion = "assert " + instantiate_prop(prop, var_names) + ', "violated postcondition"'
            assertion_ast = ast.parse(assertion)

            if assertion.find('return_value') >= 0:
                postconditions += assertion_ast.body

        return postconditions
    
    def insert_assertions(self, body):
        new_body = super().insert_assertions(body)
        postconditions = self.postconditions()

        body_ends_with_return = isinstance(new_body[-1], ast.Return)
        if body_ends_with_return:
            saver = "return_value = " + astunparse.unparse(new_body[-1].value)
        else:
            saver = "return_value = None"
    
        saver_ast = ast.parse(saver)
        postconditions = [saver_ast] + postconditions

        if body_ends_with_return:
            return new_body[:-1] + postconditions + [new_body[-1]]
        else:
            return new_body + postconditions

In [ ]:
with InvariantAnnotator() as annotator:
    my_sqrt(5)

In [ ]:
my_sqrt_def = annotator.functions_with_invariants()

In [ ]:
print_content(my_sqrt_def, '.py')

In [ ]:
exec(my_sqrt_def)

In [ ]:
with ExpectError():
    my_sqrt(-1)

In [ ]:
with InvariantAnnotator() as annotator:
    y = sum3(3, 4, 5)
    y = sum3(-3, -4, -5)
    y = sum3(0, 0, 0)

In [ ]:
print_content(annotator.functions_with_invariants(), '.py')

In [ ]:
def list_length(L):
    if L == []:
        length = 0
    else:
        length = 1 + list_length(L[1:])
    return length

In [ ]:
with InvariantAnnotator() as annotator:
    length = list_length([1, 2, 3])

In [ ]:
print_content(annotator.functions_with_invariants(), '.py')

In [ ]:
def print_sum(a, b):
    print(a + b)

In [ ]:
with InvariantAnnotator() as annotator:
    print_sum(31, 45)

In [ ]:
print_content(annotator.functions_with_invariants(), '.py')

Exercise: Have `InvariantTransformer` save initial values and take care of multiple returns.

Exercise: Implement implications

## Mining Specifications from Generated Tests

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```markdown
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_